<a href="https://colab.research.google.com/github/maurilanzo/Data-Science-Lab/blob/main/ds-labs/lab5/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setting

In [22]:
import random
import numpy as np
import pandas as pd
import torch
import sklearn as sk
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [23]:
!git clone https://github.com/maurilanzo/Data-Science-Lab.git

Cloning into 'Data-Science-Lab'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 76 (delta 22), reused 26 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (76/76), 23.05 MiB | 15.42 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [24]:
%cd Data-Science-Lab/ds-labs/lab5/

/content/Data-Science-Lab/ds-labs/lab5/Data-Science-Lab/ds-labs/lab5


In [25]:
SEED = 42
N = 100
random.seed(SEED)
np.random.seed(SEED)

torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

indices = list(range(N))
values = np.random.rand(N).tolist()
noise = torch.randn(N).numpy()

#Data Preprocessing

In [26]:
X = pd.read_csv("Lab 05 - Regression Exam Simulation/train_dataset.csv",usecols=range(58))
y = pd.read_csv("Lab 05 - Regression Exam Simulation/train_dataset.csv",usecols=[58])
X_test = pd.read_csv("Lab 05 - Regression Exam Simulation/test_dataset.csv",usecols=range(58))

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=42)
#X_train = X.copy()
#y_train = y.copy()

#Search some useless columns or rows

#print(X_val.nunique())    #DROPPING COLUMNS WITH 1 VALUE, TRY ALSO WITH 2 VALUES


#fill float nan values
float_cols = X_train.select_dtypes(include=float).columns
train_median = X_train[float_cols].median()
train_means = X_train[float_cols].mean()
X_train[float_cols]= X_train[float_cols].fillna(train_means)

float_cols = X_val.select_dtypes(include=float).columns
X_val[float_cols]= X_val[float_cols].fillna(train_means)

float_cols = X_test.select_dtypes(include=float).columns
X_test[float_cols]= X_test[float_cols].fillna(train_means)

#transform ordinal values and fill nan
ord_cols = [f'ord_{i}' for i in range(20)]
enc = OrdinalEncoder()

X_train.loc[:,"ord_0":"ord_19"] = enc.fit_transform(X_train.loc[:,"ord_0":"ord_19"])
train_ord_mean=X_train.loc[:,"ord_0":"ord_19"].mean()
X_train.loc[:,"ord_0":"ord_19"] = X_train.loc[:,"ord_0":"ord_19"].fillna(train_ord_mean)      #ATTENTION: Try with mean and median

X_val.loc[:,"ord_0":"ord_19"] = enc.transform(X_val.loc[:,"ord_0":"ord_19"])
X_val.loc[:,"ord_0":"ord_19"] = X_val.loc[:,"ord_0":"ord_19"].fillna(train_ord_mean)      #ATTENTION: Try with mean and median

X_test.loc[:,"ord_0":"ord_19"] = enc.transform(X_test.loc[:,"ord_0":"ord_19"])
X_test.loc[:,"ord_0":"ord_19"] = X_test.loc[:,"ord_0":"ord_19"].fillna(train_ord_mean)      #ATTENTION: Try with mean and median

#convert categorical values


cat_cols = [f'cat_{i}' for i in range(8)]
X_train=pd.get_dummies(X_train,dummy_na=True,columns=cat_cols)
X_val=pd.get_dummies(X_val,dummy_na=True,columns=cat_cols)
X_test=pd.get_dummies(X_test,dummy_na=True,columns=cat_cols)

#Now we can proceed with standardization
scaler = StandardScaler()
X_train.loc[:,"cont_0":"ord_19"] = scaler.fit_transform(X_train.loc[:,"cont_0":"ord_19"])
X_val.loc[:,"cont_0":"ord_19"] = scaler.transform(X_val.loc[:,"cont_0":"ord_19"])
X_test.loc[:,"cont_0":"ord_19"] = scaler.transform(X_test.loc[:,"cont_0":"ord_19"])



/tmp/ipython-input-2155974252.py:32: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train.loc[:,"ord_0":"ord_19"] = X_train.loc[:,"ord_0":"ord_19"].fillna(train_ord_mean)      #ATTENTION: Try with mean and median
/tmp/ipython-input-2155974252.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_val.loc[:,"ord_0":"ord_19"] = X_val.loc[:,"ord_0":"ord_19"].fillna(train_ord_mean)      #ATTENTION: Try with mean and median
/tmp/ipython-input-2155974252.py:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will ch

#Hyperparameters tuning

In [27]:
#We can start tuning
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid


In [28]:
y_train = y_train.values.ravel()
y_val = y_val.values.ravel()

##Polinomial regression

In [29]:



model = make_pipeline(
    PolynomialFeatures(degree=2),
    LinearRegression()
)
model.fit(X_train, y_train)
val_pred = model.predict(X_val)
score = r2_score(y_val, val_pred)

print("Validation R²:", score)

"""# Retrain on train+val and test
final_model = make_pipeline(PolynomialFeatures(degree=best_params['degree']), LinearRegression())
final_model.fit(np.concatenate([X_train, X_val]), np.concatenate([y_train, y_val]))
y_test_pred = final_model.predict(X_test)
print("Test R²:", r2_score(y_test, y_test_pred))"""

Validation R²: -8.379287574299823


'# Retrain on train+val and test\nfinal_model = make_pipeline(PolynomialFeatures(degree=best_params[\'degree\']), LinearRegression())\nfinal_model.fit(np.concatenate([X_train, X_val]), np.concatenate([y_train, y_val]))\ny_test_pred = final_model.predict(X_test)\nprint("Test R²:", r2_score(y_test, y_test_pred))'

##Random forest

In [31]:

param_grid_rf = {
    'n_estimators': [250,300],
    'max_depth': [None]
}
best_score = -float('inf')
best_params = None

for params in ParameterGrid(param_grid_rf):
    model = RandomForestRegressor( **params)
    model.fit(X_train, y_train)
    val_pred = model.predict(X_val)
    score = r2_score(y_val, val_pred)

    if score > best_score:
        best_score = score
        best_params = params

print("Best RF params:", best_params)
print("Validation R²:", best_score)

Best RF params: {'max_depth': None, 'n_estimators': 250}
Validation R²: 0.8101555095777345


##Ridge Regression

In [32]:
param_grid_ridge = {"alpha" : [0.1, 0.2],
"fit_intercept" : [True, False]}

best_score = -float('inf')
best_params = None

for params in ParameterGrid(param_grid_ridge):
    model = make_pipeline(StandardScaler(), Ridge(**params))
    model.fit(X_train, y_train)
    val_pred = model.predict(X_val)
    score = r2_score(y_val, val_pred)

    if score > best_score:
        best_score = score
        best_params = params

print("Best Ridge params:", best_params)
print("Validation R²:", best_score)



Best Ridge params: {'alpha': 0.2, 'fit_intercept': False}
Validation R²: 0.26450361236454234


##Lasso Regression

In [33]:
param_grid_lasso = {'alpha': [0.001, 0.01, 0.1, 1, 10]}

best_score = -float('inf')
best_params = None

for params in ParameterGrid(param_grid_lasso):
    model = make_pipeline(StandardScaler(), Lasso(max_iter=10000, **params))
    model.fit(X_train, y_train)
    val_pred = model.predict(X_val)
    score = r2_score(y_val, val_pred)

    if score > best_score:
        best_score = score
        best_params = params

print("Best Lasso params:", best_params)
print("Validation R²:", best_score)

Best Lasso params: {'alpha': 0.01}
Validation R²: 0.2691758140460144


#Best: Random forest with: Best RF params: {'max_depth': None, 'n_estimators': 250}


In [35]:
params = {'max_depth': None, 'n_estimators': 250}
model = RandomForestRegressor( **params)
model.fit(X_train, y_train)
predictions = model.predict(X_val)


In [36]:
submission = pd.DataFrame({
    "index": range(len(predictions)),
    "value": predictions
})
submission_path = "submission.csv"
submission.to_csv(submission_path, index=False)